In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [9]:

# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_in_Los_Angeles").text

In [10]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [54]:
neighborhoodList = []
# append the data into the list
#for row in soup.find_all("div", class_="div-col columns column-width")[1].findAll("li"):
 #   neighborhoodList.append(row.text)
    
L = len(soup.find_all("div", class_="div-col columns column-width"))
for i in range(L):
    for row in soup.find_all("div", class_="div-col columns column-width")[i].findAll("li"):
        neighborhoodList.append(row.text)

for i in range(len(neighborhoodList)):
    for num, char in enumerate(neighborhoodList[i]):
        if char == "[":
            pos1 = num
            neighborhoodList[i] = neighborhoodList[i][:num]

In [55]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})

df.head()

,Neighborhood
0,Angelino Heights
1,Angeles Mesa
2,Angelus Vista
3,Arleta
4,Arlington Heights


In [57]:
df.shape

(200, 1)

In [63]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="Capstone Project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of L.A. are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of L.A. are 34.0536909, -118.242766.


In [68]:
# create map of Toronto using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

map_la

In [69]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles, CA'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [75]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [77]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [85]:
df.head()

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070290,-118.254800
1,Angeles Mesa,2.421100,-76.917380
2,Angelus Vista,34.087575,-118.267156
3,Arleta,34.249050,-118.433490
4,Arlington Heights,34.039890,-118.325160


In [80]:
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_la)  
    
map_la

In [83]:
# define Foursquare Credentials and Version
CLIENT_ID = 'EBHWERUV3ZESXQRZ122L1410MYXWDEBBXEDDFXT321AMBLGP' # your Foursquare ID
CLIENT_SECRET = 'UCSDN52ZOIQLY0M3UTBH3R4XY1YTLTPD0GRF0OMAKUVV3OJ4' # your Foursquare Secret
VERSION = '20201115' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EBHWERUV3ZESXQRZ122L1410MYXWDEBBXEDDFXT321AMBLGP
CLIENT_SECRET:UCSDN52ZOIQLY0M3UTBH3R4XY1YTLTPD0GRF0OMAKUVV3OJ4


In [89]:
radius = 500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [91]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4573, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Angelino Heights,34.07029,-118.2548,Halliwell Manor,34.069329,-118.254165,Performing Arts Venue
1,Angelino Heights,34.07029,-118.2548,Guisados,34.070262,-118.250437,Taco Place
2,Angelino Heights,34.07029,-118.2548,Eightfold Coffee,34.071245,-118.250698,Coffee Shop
3,Angelino Heights,34.07029,-118.2548,"Michael Jackson's ""Thriller"" House (and Tree)",34.069557,-118.254599,Historic Site
4,Angelino Heights,34.07029,-118.2548,Ototo,34.072659,-118.251740,Sake Bar


In [92]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Angelino Heights,23,23,23,23,23,23
Angelus Vista,10,10,10,10,10,10
Arleta,4,4,4,4,4,4
Arlington Heights,12,12,12,12,12,12
Arts District,44,44,44,44,44,44
Atwater Village,37,37,37,37,37,37
Baldwin Hills,12,12,12,12,12,12
Baldwin Hills/Crenshaw,41,41,41,41,41,41
Baldwin Village,7,7,7,7,7,7


In [93]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 372 uniques categories.


In [94]:

# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Performing Arts Venue', 'Taco Place', 'Coffee Shop',
       'Historic Site', 'Sake Bar', 'Art Gallery', 'Japanese Restaurant',
       'BBQ Joint', 'Bakery', 'Cocktail Bar', 'Breakfast Spot',
       'Convenience Store', 'Pizza Place', 'Motel', 'Market',
       'Clothing Store', 'Hotel', 'Trail', 'Mexican Restaurant',
       'Grocery Store', 'Sculpture Garden', 'Pet Store',
       'American Restaurant', 'Ice Cream Shop', 'Seafood Restaurant',
       'Italian Restaurant', 'Rock Club', 'Furniture / Home Store',
       'Thai Restaurant', 'Arts & Entertainment', 'Athletics & Sports',
       'Restaurant', 'Liquor Store', 'Miscellaneous Shop',
       'Korean Restaurant', 'Home Service', 'Kitchen Supply Store',
       'Beer Garden', 'Mediterranean Restaurant', 'Brewery',
       'Climbing Gym', 'Pie Shop', 'German Restaurant', 'Arcade',
       'Smoothie Shop', 'Café', "Men's Store", 'Bookstore',
       'Vegetarian / Vegan Restaurant', 'Sushi Restaurant'], dtype=object)

In [95]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [96]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(4573, 373)


,Neighborhoods,ATM,Acai House,Accessories Store,Adult Boutique,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Library,College Quad,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Piano Bar,Picnic Area,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Post Office,Print Shop,Pub,Public Art,Ramen Restaurant,Record Sho

In [97]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(196, 373)


,Neighborhoods,ATM,Acai House,Accessories Store,Adult Boutique,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Library,College Quad,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Piano Bar,Picnic Area,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Post Office,Print Shop,Pub,Public Art,Ramen Restaurant,Record Sho

In [98]:

len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

14

In [99]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [100]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Angelino Heights,0.0
1,Angelus Vista,0.0
2,Arleta,0.0
3,Arlington Heights,0.0
4,Arts District,0.0


In [101]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [103]:
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [104]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Angelino Heights,0.0,0
1,Angelus Vista,0.0,0
2,Arleta,0.0,0
3,Arlington Heights,0.0,0
4,Arts District,0.0,0


In [110]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")


print(kl_merged.shape)
kl_merged.head() # check the last columns!


(196, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Angelino Heights,0.0,0,34.07029,-118.25480
123,Panorama City,0.0,0,34.22302,-118.44875
124,Park La Brea,0.0,0,34.06257,-118.35774
125,Picfair Village,0.0,0,34.04500,-118.36528
126,Pico Robertson,0.0,0,34.05575,-118.38000


In [111]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(196, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Angelino Heights,0.000000,0,34.070290,-118.254800
24,Bunker Hill,0.000000,0,34.052020,-118.250360
23,Broadway-Manchester,0.000000,0,33.995234,-118.467059
1,Angelus Vista,0.000000,0,34.087575,-118.267156
2,Arleta,0.000000,0,34.249050,-118.433490
3,Arlington Heights,0.000000,0,34.039890,-118.325160
4,Arts District,0.000000,0,34.041964,-118.236387
5,Atwater Village,0.000000,0,34.119700,-118.258870
6,Baldwin Hills,0.000000,0,34.021570,-118.367650
8,Baldwin Village,0.000000,0,34.070445,-118.200710


In [112]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [113]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [114]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Angelino Heights,0.0,0,34.070290,-118.254800
24,Bunker Hill,0.0,0,34.052020,-118.250360
23,Broadway-Manchester,0.0,0,33.995234,-118.467059
1,Angelus Vista,0.0,0,34.087575,-118.267156
2,Arleta,0.0,0,34.249050,-118.433490
3,Arlington Heights,0.0,0,34.039890,-118.325160
4,Arts District,0.0,0,34.041964,-118.236387
5,Atwater Village,0.0,0,34.119700,-118.258870
6,Baldwin Hills,0.0,0,34.021570,-118.367650
8,Baldwin Village,0.0,0,34.070445,-118.200710


In [115]:

kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
36,Chinatown,0.017857,1,34.062290,-118.238370
111,Naud Junction,0.017544,1,34.061070,-118.235440
38,Crenshaw,0.020408,1,34.010710,-118.335120
77,Hollywood,0.021739,1,34.101560,-118.337080
158,Toy District,0.013514,1,34.048025,-118.258447
42,Downtown,0.011364,1,34.048340,-118.255640
120,Pacific Palisades,0.021277,1,34.047100,-118.526880
16,Beverly Hills Post Office,0.020000,1,34.070161,-118.399227
141,Silver Lake,0.016393,1,34.082730,-118.273070
98,Little Tokyo,0.020000,1,34.049392,-118.240579


In [116]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
188,Wholesale District,0.058824,2,34.03122,-118.251867
